# Execute & Format Control

## Execute

### Definition

_Execute_ is a monadic function represented by `⍎`, which you can type with <kbd>APL</kbd>+<kbd>;</kbd>; its dyadic use will be explained in the Specialist's Section at the end of this chapter.

_Execute_ takes a character vector (or scalar) as its argument.

If the character vector represents a valid APL expression, _Execute_ will just... execute it, as if it had been typed on the keyboard.
Otherwise, an error will be reported.

Take a look at the following example:

In [1]:
⎕← letters ← '5×6+2'

5×6+2

In [2]:
⍎letters

40

The argument can contain any valid expression:
 - numeric or character constants, or variables;
 - left arrows (assignment) or right arrows (branch);
 - primitive or defined functions and operators; and
 - calls to other _execute_ functions.

Let us define a short function to call from within _execute_:

In [3]:
∇ r ← x Plus y
    r ← x + y
∇

In the expression below, _execute_ calls our `Plus` function and creates a new variable:

In [4]:
⍎'new ← 3 + 4 Plus 5'

In [5]:
new

12

If the expression returns a result, it will be used as the result of _execute_:

In [6]:
res ← ⍎'3 Plus 10'

In [7]:
res

13

We could just as well have written

In [8]:
⍎'res ← 3 Plus 10'

In [9]:
res

13

<!--begin beware style=warning-->
***Beware***:

 > Note that if the argument does not return a result, it can still be executed, but _execute_ will not return a result, and any attempt to assign it to a variable or to use it in any other way will cause a `VALUE ERROR`.
<!--end-->

Take a modification of our `Plus` function that returns no result:

In [10]:
∇ x PlusNoRes y;r
    r ← x+y
∇

These expressions all work:

In [11]:
⍎''
⍎'     '
⍎'3 PlusNoRes 5'

But trying to assign results from all those expressions will fail,

In [12]:
res ← ⍎''
res ← ⍎'     '
res ← ⍎'3 PlusNoRes 5'

VALUE ERROR: No result was provided when the context expected one
      res←⍎''
          ∧
VALUE ERROR: No result was provided when the context expected one
      res←⍎'     '
          ∧
⍎VALUE ERROR: No result was provided when the context expected one
      3 PlusNoRes 5
        ∧


because that is equivalent to writing the following:

In [13]:
res ← 
res ←       
res ← 3 PlusNoRes 5

SYNTAX ERROR: Missing right argument
      res←
         ∧
SYNTAX ERROR: Missing right argument
      res←
         ∧
VALUE ERROR: No result was provided when the context expected one
      res←3 PlusNoRes 5
            ∧


### Some Typical Uses

#### Convert Text into Numbers

_Execute_ may be used to convert characters into numbers.
One common application of execute is to convert numeric data, stored as character strings in a text file (for example, a `.csv` file), into binary numbers.
You can just read in a string such as `'123, 456, 789'` and execute it to obtain the corresponding 3-item vector:

In [14]:
⍎'123, 456, 789'

123 456 789

We saw in [the chapter about user-defined functions](./User-Defined-Functions.ipynb) that _format_ can be used to convert numbers to characters; the reverse can be done using _execute_.
This explains why those two functions are represented by "reversed" symbols, as shown in <!--figure-->the figure below<!--Dual_Execute_Format-->:

![Representation of the duality between the _execute_ and _format_ primitives.](res/Dual_Execute_Format.png)

There is, however, a major difference: _format_ can be applied to matrices, whereas _execute_ can only be applied to vectors.

In [15]:
birthdate ← 'October 14th, 1952'
+/ ⍎ ⎕←birthdate[9 10,13+⍳5]

14 1952

1966

Notice that the `'14 1952'` above is a character _vector_ of length 7, and not a vector of 2 character vectors.
In fact, compare

In [16]:
birthdate[9 10,13+⍳5]

14 1952

In [17]:
≢birthdate[9 10,13+⍳5]

7

with

In [18]:
'14' '1952'

┌──┬────┐
│14│1952│
└──┴────┘

to which `⍎` _cannot_ be applied:

In [19]:
⍎'14' '1952'

DOMAIN ERROR
      ⍎'14' '1952'
      ∧


Because _execute_ can only be applied to vectors, a matrix of numeric characters can only be converted after it has been ravelled.
But to avoid characters of one row being attached to those of the previous row, it is necessary to catenate a blank character before ravelling.

As an example, take the matrix `mat` below:

In [20]:
⎕← mat ← 4 4⍴' 8451237 9332607'

845
1237
 933
2607

If we ravel it and execute it, we get

In [21]:
⍎,mat

8451237 9332607

which is not what we want.
The correction conversion will be obtained by first catenating a blank space:

In [22]:
⍎,mat,' '

845 1237 933 2607

#### A Safer and Faster Solution

Using _execute_ to convert characters into numbers may cause errors if the characters do not represent valid numbers.
So, we strongly recommend that you instead use `⎕VFI` (for _Verify and Fix Input_).
This is a specialised _system function_ that performs the same conversion, but securely, and is about twice as fast as _execute_.
`⎕VFI` will be studied in [the chapter about system interfaces](./System-Interfaces.ipynb).

#### Other Uses

_Execute_ can be used for many other purposes, including some that may be considered to be rather advanced programming techniques.
Some examples are provided in the Specialist's Section at the end of this chapter:
 - conditional execution (rather obsolete);
 - case selection (also obsolete);
 - dynamic variable creation.

Please bear in mind that these _execute_ use-cases aren't necessarily _recommended_ programming practices.

### Make Things Simple

The vector passed in to _execute_ is often constructed by catenating pieces of text, or tokens.

These tokens may contain quotes (which must then be doubled), commas, parentheses, etc.
But to build the final expression, you will also need quotes (to delimit the tokens), commas (to concatenate them), parentheses, and so on.

By now, the expression is becoming extremely complex.
It may be difficult to see if a comma is part of a token or is being used to concatenate two successive tokens, and this is only partly alleviated by syntax colouring that modern IDEs provide.
It may be hard to see whether or not the parentheses and quotes are properly balanced.
If the final expression is wrong, fixing it might be difficult, and if it is correct, later modifying it or expanding it might be just as difficult.

To simplify maintenance, it is good practice to assign the text to a variable before executing it.
If the operation fails, for any reason, you can just display the variable to see if it looks correct.
For example, here is a statement involving _execute_:

In [23]:
size ← 43
⍎'tab',(⍕size),'←(4 ',(⍕size),'⍴'') '''

⍎SYNTAX ERROR
      tab43←(4 43⍴') '
                 ∧


That's rather obscure!
If any problem occurs, it can be difficult to spot the cause.

Let us insert a variable just before the _execute_ function:

In [24]:
⍎debug←'tab',(⍕size),'←(4 ',(⍕size),'⍴'') '''

⍎SYNTAX ERROR
      tab43←(4 43⍴') '
                 ∧


Now, it easy to look at `debug` and see if its value is what we would expect:

In [25]:
debug

tab43←(4 43⍴') '

Obviously, this is not a correct statement, so it failed when we tried to execute it.

## The Format Primitive

The _format_ primitive function `⍕`, typed with <kbd>APL</kbd>+<kbd>'</kbd>, has already been briefly described in [a previous section](./User-Defined-Functions.ipynb#Format).
We shall cover it in more depth in this section.

### Monadic Format

Monadic _format_ converts any array, whatever its value, into its character representation.
This applies to numbers, characters, and nested arrays.
The result is exactly the same as you would see if you displayed the array on your screen, because APL internally uses monadic _format_ to display arrays.
The previous statement assumes that you have no options modifying your output; for example, if you have `]box on` it is no longer true that _format_ produces exactly the same representation as if you just displayed the the array yourself:

In [26]:
⍕1 (2 3)

1 2 3

In [27]:
]box

Is ON

In [28]:
1 (2 3)

┌─┬───┐
│1│2 3│
└─┴───┘

Ignoring the effects of such session modifiers like `]box`, which will be covered with some more detail later in this chapter (see [this section](#Output-User-Commands)), monadic _format_ is such that:
 - character arrays are not converted, they remain unchanged; and
 - numeric and nested arrays are converted into vectors or matrices of characters.

In [29]:
⎕←chemistry ← 3 5⍴'H2SO4CaCO3Fe2O3'

H2SO4
CaCO3
Fe2O3

`chemistry` is a character matrix with shape `3 5`, and it is not modified by `⍕`:

In [30]:
⍴⎕←⍕chemistry

H2SO4
CaCO3
Fe2O3

3 5

In [31]:
chemistry≡⍕chemistry

1

On the other hand, a numeric vector with, say, 3 items, becomes a (longer) character vector once converted:

In [32]:
≢52 69 76

3

In [33]:
≢⎕←⍕52 69 76

52 69 76

8

A nested matrix like `nesMat`,

In [34]:
⍴⎕←nesMat ← 2 3 ⍴ 'Dyalog' 44 'Hello' 27 (2 2 ⍴ 8 6 2 4) (2 3⍴1 2 0 0 0 5)

┌──────┬───┬─────┐
│Dyalog│44 │Hello│
├──────┼───┼─────┤
│27 │8 6│1 2 0│
│ │2 4│0 0 5│
└──────┴───┴─────┘

2 3

which we have already used before, becomes a character matrix that is 20 characters wide and with 3 rows, because `nesMat` contained two small matrices:

In [35]:
⍴⎕←⍕nesMat

Dyalog 44 Hello 
 27 8 6 1 2 0 
 2 4 0 0 5 

3 20

### Dyadic Format

#### Definition

Dyadic _format_ applies **only** to numeric values; any attempt to apply it to characters will cause a `DOMAIN ERROR`.

The general syntax of _format_ is `descriptor⍕values`,
where `values` can be an array of any _rank_.
Dyadic _format_ converts numbers into text in a format that is described by the left argument, the format _descriptor_.
`descriptor` is therefore made up of two numbers:
 - the first number indicates the number of characters to be assigned to each numeric value; or to put it another way, the width of the field in which each numeric value is to be represented; and
 - the second number indicates how many decimal digits will be displayed.

In [36]:
⎕RL ← 73
⎕←nm ← (?3 4⍴200000)÷100

21.21 1534.88 375.46 704.5 
1125.14 1963.52 464.45 1438.25
 796.53 1569 157.14 886.59

The representation above is the normal display of the `nm` matrix, and it is also how monadic _format_ would present the matrix.

Compare that with the result below, where we represent each number right-aligned in a field that is 8 characters wide, with 2 decimal digits.

In [37]:
8 2⍕nm

21.21 1534.88 375.46 704.50
 1125.14 1963.52 464.45 1438.25
 796.53 1569.00 157.14 886.59

The result has, of course, 3 rows and 32 columns (8 characters for each of the 4 columns):

In [38]:
⍴8 2⍕nm

3 32

We can also draw a basic ruler (with the help of a short dfn) below the formatted matrix to help you count the width of each field:

In [39]:
]dinput
ruler ← {
    c ← ¯1↑⍴⍵
    ⍵⍪c⍴4 1 4 1/'¯''¯|'
}

In [40]:
ruler 8 2⍕nm

21.21 1534.88 375.46 704.50
 1125.14 1963.52 464.45 1438.25
 796.53 1569.00 157.14 886.59
¯¯¯¯'¯¯¯¯|¯¯¯¯'¯¯¯¯|¯¯¯¯'¯¯¯¯|¯¯

In the next example we represent each number in a field that is 6 characters wide, right-aligned, and with no decimal digits.

In [41]:
ruler 6 0⍕nm

21 1535 375 705
 1125 1964 464 1438
 797 1569 157 887
¯¯¯¯'¯¯¯¯|¯¯¯¯'¯¯¯¯|¯¯¯¯

The result has now 3 rows and 24 columns:

In [42]:
⍴6 0⍕nm

3 24

<!--begin remark-->
***Remark***:

 > You can see that the numbers to be formatted are **rounded** rather than truncated when the specified format does not allow the full precision of the numbers to be shown.
<!--end-->

#### Overflow

If a column is not wide enough to represent some of the numbers, these numbers will be replaced by asterisks.

Recall what `nm` looked like:

In [43]:
nm

21.21 1534.88 375.46 704.5 
1125.14 1963.52 464.45 1438.25
 796.53 1569 157.14 886.59

We have seen above that `8 2⍕nm` looked good:

In [44]:
8 2⍕nm

21.21 1534.88 375.46 704.50
 1125.14 1963.52 464.45 1438.25
 796.53 1569.00 157.14 886.59

If we reduce the width of the columns by 1, some values will now be adjacent to the values immediately to their left, making them difficult to read. For example, the second column will be adjacent to the left column:

In [45]:
7 2⍕nm

21.211534.88 375.46 704.50
1125.141963.52 464.451438.25
 796.531569.00 157.14 886.59

If we further reduce the width of the columns, the largest values will no longer fit in their allotted space and will be replaced by asterisks. Most of the other numbers are now adjacent to their neighbours.

In [46]:
6 2⍕nm

21.21******375.46704.50
************464.45******
796.53******157.14886.59

<!--begin remark-->
***Remark***:

 > To calculate the width required to represent a number you must account for the minus sign, the integer digits, the decimal point, and as many decimal digits as specified in the _descriptor_.

#### Multiple Specifications

One can define a different format for each column of numbers.
Each format definition is made of 2 numbers, so if the matrix has `n` columns, the left argument must have `2×n` items:

In [47]:
ruler 8 2 10 0 9 4 8 2⍕nm

21.21 1535 375.4600 704.50
 1125.14 1964 464.4500 1438.25
 796.53 1569 157.1400 886.59
¯¯¯¯'¯¯¯¯|¯¯¯¯'¯¯¯¯|¯¯¯¯'¯¯¯¯|¯¯¯¯'

In this case, we formatted the first column with `8 2`, the second column with `10 0`, the third column with `9 4`, and the fourth column with `8 2` again.

If the format descriptor (the left argument) does not contain enough pairs of values, it will be repeated as many times as needed, provided that the width of the matrix is a multiple of the number of pairs.

In other words, in `desc⍕values`, the residue `(≢desc)|2×¯1↑⍴values` must be equal to 0, otherwise a `LENGTH ERROR` is reported.
For example,

In [148]:
desc ← 8 2 10 0
desc⍕nm

21.21 1535 375.46 705
 1125.14 1964 464.45 1438
 796.53 1569 157.14 887

In the above, columns 3 and 4 reused the _descriptors_ of columns 1 and 2, respectively `8 2` and `10 0`.
This is equivalent to writing the repeated pairs by hand:

In [150]:
8 2 10 0 8 2 10 0⍕nm

21.21 1535 375.46 705
 1125.14 1964 464.45 1438
 796.53 1569 157.14 887

We can also compare `desc⍕nm` with the statement above about the residue:

In [151]:
(≢desc)|2×¯1↑⍴nm

0

#### Scientific Representation

If the second item of a pair of _format_ descriptors is negative, for example in `9 ¯3⍕12345`, numbers are formatted in _scientific_ notation (also described as _exponential_ notation; see [a previous Specialist's Section](./Data-and-Variables.ipynb#Representation-of-Numbers)), with as many significant digits in the mantissa as specified by the descriptor:

In [157]:
8 ¯2⍕nm

2.1E1 1.5E3 3.8E2 7.0E2 
 1.1E3 2.0E3 4.6E2 1.4E3 
 8.0E2 1.6E3 1.6E2 8.9E2

In [158]:
8 ¯4⍕nm

2.121E1 1.535E3 3.755E2 7.045E2 
1.125E3 1.964E3 4.645E2 1.438E3 
7.965E2 1.569E3 1.571E2 8.866E2

You can refer to [Wikipedia's article on "Scientific Notation"](https://en.wikipedia.org/wiki/Scientific_notation#E_notation) for more information on how scientific notation works.

#### Scalar Descriptor

When the *descriptor* is reduced to a simple scalar, the columns are formatted in the smallest width compatible with the values they contain, plus one separating space.
The scalar descriptor is then used to specify the number of decimal places to use, or the number of significant digits to use in the mantissa of the scientific representation of the numbers, depending on the sign of the descriptor.

For example,

In [153]:
2⍕nm

21.21 1534.88 375.46 704.50
 1125.14 1963.52 464.45 1438.25
 796.53 1569.00 157.14 886.59

tells `⍕` to use 2 decimal places for all numbers, whereas

In [160]:
¯3⍕nm

2.12E1 1.53E3 3.75E2 7.05E2
 1.13E3 1.96E3 4.64E2 1.44E3
 7.97E2 1.57E3 1.57E2 8.87E2

tells `⍕` to use 3 digits in the mantissa of all numbers.

In both examples above we see that each column is separated from the preceding one (and from the left margin too!) by a single space.

This technique is convenient for experimental purposes, to have the most compact presentation possible, but you cannot control the total width of the final result with it.

## The `⎕FMT` System Function

The _format_ primitive function is inadequate for producing professional looking output, such as one may require for common business purposes, because:

 - negative values are represented by a high minus sign, which is rather unusual outside the APL world;
 - a large value, like 5244172.68, is displayed in a single unpleasant block, while it would look better with thousands separators, like this: 5,244,172.68;
 - national conventions differ from one country to another. I would be convenient if the value shown above could be written as 5,244,172.68, or 5 244 172,68, or 5.244.172,68; and
 - it would be nice if negative values could have different styles of presentation, depending on the usage and the context: -427 or (427).

For all these reasons, the `⍕` primitive is sometimes inappropriate, and it is better to use a _system function_ named `⎕FMT` (where `FMT` also stands for _format_).

### Monadic Use

Monadic `⎕FMT`, like its primitive counterpart, converts numbers into text, without any specific control over the formatting.

The result of `⎕FMT` is **always** a matrix, even if it is applied to a numeric scalar or vector.
This is different from `⍕`:

In [52]:
⍴⍕ 523 12 742

10

In [53]:
⍴⎕FMT 523 12 742

1 10

The general presentation is the same, except for some very special cases.

### Dyadic Use

#### Overview

Like `⍕`, dyadic `⎕FMT` accepts a _descriptor_ for its left argument: `descriptor ⎕FMT values`.

The right argument `values` can be:
 - a scalar, a vector, a matrix, but, unlike `⍕`, **not** a higher rank array; or
 - a nested scalar or a nested vector, whose items are simple arrays (not nested) or rank not greater than 2.

If `values` is a nested vector, each of its items must be homogeneous (either character or numeric).
In other words, an item of `values` may not itself be of mixed type.

The `descriptor` argument is a character vector, made of a succession of elementary descriptors separated by commas; for example:

In [54]:
'I6,4A2,4F8.2' ⎕FMT (12 3 4)(2 4⍴'ABCDEFGH')nm

12 A B C D 21.21 1534.88 375.46 704.50
 3 E F G H 1125.14 1963.52 464.45 1438.25
 4 796.53 1569.00 157.14 886.59

Each elementary descriptor is made up of:
 - a letter, the _specification_, specifying the data representation (integer, decimal, character);
 - numeric values which specify the width and the shape of the result;
 - _qualifiers_ and _affixtures_, used to specify further the details of the formatting; and
 - sometimes a _repetition factor_, to apply the same description to several columns.

These elementary descriptors are used one after the other, from left to right, and applied to successive values (or columns of values).

Usually each array specified on the right has its own specific descriptor on the left.

For example, in the example statement above,
 - `'I6'` applies to the vector `12 3 4`;
 - `'4A2'` applies to the four columns of the character matrix `2 4⍴ABCDEFGH`; and
 - `'F8.2'` applies to `nm`.

However, an elementary descriptor can apply to several arrays if they are to share the same formatting, or a single array can require several descriptors when each of its columns is to be formatted differently.

Matrices are formatted normally, whereas vectors are transposed (vertically) into columns.

#### Specifications I and F

These specifications are used to display numeric values:
 - `'I'` for **I**ntegers; and
 - `'F'` for **F**ractional.

They use the following syntax:
 - `'rIw'`
   - `'w'` is the **w**idth (the number of characters) dedicated to each column of numbers; and
   - `'r'` is the number of columns to which this format specification is to be applied (this is the **r**epetition factor mentioned earlier.
 - `'rFw.d'`
   - `'w'` is the **w**idth (the number of characters) dedicated to each column of numbers;
   - `'d'` is the number of **d**ecimal digits to display; and
   - `'r'` is the **r**epetition factor.

Notice that the role of `'w.d'` in the specification `'F'` is related to the syntax of the left argument of the primitive function _format_.

Let us work on the numeric matrix `nm`,

In [55]:
nm

21.21 1534.88 375.46 704.5 
1125.14 1963.52 464.45 1438.25
 796.53 1569 157.14 886.59

and on the following `price` vector:

In [56]:
price ← 5.2 11.5 3.6 4 8.45

In [57]:
'I4,2F9.1,I8,F6.1' ⎕FMT nm price

21 1534.9 375.5 705 5.2
1125 1963.5 464.5 1438 11.5
 797 1569.0 157.1 887 3.6
 4.0
 8.5

Here is an explanation of the descriptors used above:
 - `'I4'`: the first column of `nm` is displayed with a width of 4 characters, as integers (the values are rounded);
 - `'2F9.1'`: two columns are displayed with a width of 9 characters, with only 1 decimal digit;
 - `'I8'`: the last column of `nm` is displayed with a width of 8 characters, as integers; and
 - `'F6.1'`: the `price` vector is displayed vertically with a width of 6 characters and 1 decimal digit.

Just like we mentioned earlier, notice that the `price` vector was displayed vertically, and just like with `⍕`, numbers are rounded (instead of truncated) when needed.
It is also worth mentioning that `⎕FMT` has no problem dealing with the fact that `price` takes up more vertical space than `nm`.

In the Specialist's Section you will see that you can also display numeric values using scientific (or **e**xponential) format, using the `'E'` specification, which is very similar to `'F'`.

#### Specification A

This specification is used to format characters (mnemonic: `'A'` from **a**lphabet) with the following syntax:

 - `'rAw'`
   - `'w'` is the **w**idth (the number of characters) dedicated to each column of characters; and
   - `'r'` is the number of columns to which this format applies (the **r**epetition factor).


Let us again use these two variables:

In [58]:
⎕← monMat ← 6 8⍴'January FebruaryMarch   April   May     June    '

January 
February
March 
April 
May 
June

In [59]:
⎕← chemistry ← 3 5⍴'H2SO4CaCO3Fe2O3'

H2SO4
CaCO3
Fe2O3

In [60]:
ruler '8A1,A4,9A1' ⎕FMT monMat chemistry

January H2SO4
February CaCO3
March Fe2O3
April 
May 
June 
¯¯¯¯'¯¯¯¯|¯¯¯¯'¯

Here is a breakdown of what is happening:
 - `'8A1'`: the 8 columns of `monMat` are displayed in 8 columns, each of which is 1 character wide;
 - `'A4'`: the first column of `chemistry`, displayed in a column that is 4 characters wide, produces a separation of 3 blanks; and
 - `'9A1'`: the subsequent columns of `chemistry` are displayed in (up to) 9 columns, each of which is 1 character wide.

<!--begin remarks-->
***Remarks***:

 > - Specifications (`'I'`, `'F'`, `'A'`, ...) must be uppercase letters (lowercase specifications cause errors); and
 > - We specified `'9A1'`, though we had only 4 remaining columns to format, with no problems; `⎕FMT` ignores excess repetition factors.
<!--end-->

But what would happen if the first descriptor was larger than necessary?

In [61]:
'10A1,A4,4A1' ⎕FMT monMat chemistry

January H2 SO4
FebruaryCa CO3
March Fe 2O3
April 
May 
June

The first descriptor in the example above, `'10A1'`, has a repetition factor of 10, meaning it will apply to 10 columns: the 8 columns from `monMat` plus the first 2 columns from `chemistry`.
The `'A1'` then displays those columns in columns that are 1 character wide.
The next column of `chemistry` is then displayed in a column that is 4 characters wide (because of `'A4'`), and the last two columns of `chemistry` are displayed in columns that are also 1 character wide (here we also specified more columns than are needed).

If we provide a single descriptor, it applies to all the columns of the right argument, like the following expression exemplifies:

In [62]:
'A3' ⎕FMT chemistry

H 2 S O 4
 C a C O 3
 F e 2 O 3

In the result above, each character is formatted in a column that is 3 characters wide and is right-justified.

#### Specification X

Suppose that we want to number the rows by displaying the characters `'123'` to the left of `chemistry`.
The following method would produce a poor presentation:

In [63]:
'A1,5A1' ⎕FMT '123' chemistry

1H2SO4
2CaCO3
3Fe2O3

Using `'6A1'` instead of `'A1,5A1'` would produce the same result.

To separate the digits on the left from the `chemistry` matrix on the right, we could specify a different format for the first column of `chemistry`:

In [64]:
'A1,A3,4A1' ⎕FMT '123' chemistry

1 H2SO4
2 CaCO3
3 Fe2O3

However, it is simpler to include a specific descriptor for the separation; this is the role of the `'X'` specification.

 - `'rXw'`
   - `'w'` is the **w**idth (the number of characters) of the blank column to insert; and
   - `'r'` is the **r**epetition factor.

For example, to insert a blank column that is 3 characters wide, we can specify

In [65]:
'A1,X3,5A1' ⎕FMT '123' chemistry

1 H2SO4
2 CaCO3
3 Fe2O3

`'X3'` and `'3X1` are synonymous, but the first description is simpler.

#### Text Inclusion Specification

It is sometimes convenient to separate two columns of the formatted result by a string of characters.
This string of characters must be inserted in the format description, embedded between a pair of delimiters.
You can choose from the following delimiters:

 - `<` and `>`;
 - `⊂` and `⊃`;
 - `¨` and `¨`;
 - `⎕` and `⎕`; or
 - `⍞` and `⍞`.

Of course, if you use a given set of delimiters, the character string cannot contain the closing delimiter.
For example, if you use `⊂ ⊃`, the character string cannot contain `⊃`; or, if you use `⎕ ⎕`, the character string cannot contain `⎕`.

Picking between any of the available options is a matter of preference, but the pairs `< >` and `⊂ ⊃` do present the advantage that they make it easier to spot where the included text starts and ends, which is particularly useful if the format description makes repeated use of text inclusion.
For that matter, for the remainder of this chapter we shall prefer `⊂ ⊃` or `< >` over the other options.

Let us use

In [76]:
⎕← rates ← 0.08 0.05 0.02

0.08 0.05 0.02

to calculate a result and display it:

In [77]:
res ← nm[;4]×rates
format ← '⊂| ⊃,5A1,⊂ | ⊃,4F8.2,⊂ ×⊃,I2,⊂% =⊃,F7.2,⊂€⊃'
format ⎕FMT chemistry nm (rates×100) res

| H2SO4 | 21.21 1534.88 375.46 704.50 × 8% = 56.36€
| CaCO3 | 1125.14 1963.52 464.45 1438.25 × 5% = 71.91€
| Fe2O3 | 796.53 1569.00 157.14 886.59 × 2% = 17.73€

<!-- (TODO) use something like `format⊆⍨≠\∨⌿'⊂⊃'∘.=format` to parse the `format` variable and extract the texts that will be included, with the caveat that it only works if there's no extra ⊂⊃ in there. -->

This format specification contains 9 descriptors.
To avoid a single long statement, it is possible to prepare the format description separately, save it in a variable, and use it later, as shown above.

#### Specification G – The Picture Code

The specifications we saw earlier (`I`, `F`, `X`, `A`) are very similar to those used in the "FORMAT" statement in a very popular scientific language, FORTRAN.
Another traditional language, COBOL, uses a different approach in its "PICTURE" statement.

The `'G'` specification in APL is very similar to the COBOL "PICTURE" statement.

In this specification, the letter `'G'` is followed by any string of characters, in which the characters `'Z'` and `'9'` represent the positions in which numeric digits are to be placed in the output.

The string is delimited by the same delimiters that we use for the text inclusion specification: `⊂ ⊃`, `< >`, `¨ ¨`, `⎕ ⎕`, or `⍞ ⍞`.

It works as follows:

 - all the values are rounded to the nearest integer (no decimal digit will be displayed);
 - each digit replaces one of the characters `'Z'` or `'9'` included in the `'G'` format string;
 - unused `'9'`'s are replaced by zeroes and unused `'Z'`'s are replaced by blanks;
 - all characters to the left of the first `'Z'` or `'9'`, or to the right of the last `'Z'` or `'9'` are reproduced verbatim; and
 - characters inserted between some `'Z'`'s or `'9'`'s are reproduced only if there are digits on both sides.

Some examples may help:

Let us describe the formatting of this matrix

In [96]:
⎕← mat ← 2 3⍴75 14 86 20 31 16

75 14 86
20 31 16

In [99]:
'3G⊂(9999) + ⊃' ⎕FMT mat

(0075) + (0014) + (0086) + 
(0020) + (0031) + (0016) +

Each descriptor `'9'` has been replaced by a digit of `mat`, or by a zero, and all the other characters have been reproduced from the model.

Let us now turn our attention to the `nm` matrix we have been playing around with:

In [90]:
nm

21.21 1534.88 375.46 704.5 
1125.14 1963.52 464.45 1438.25
 796.53 1569 157.14 886.59

In [91]:
'4G⊂ 9999⊃'⎕FMT nm

0021 1535 0375 0705
 1125 1964 0464 1438
 0797 1569 0157 0887

Here, each value is padded by leading zeroes and the decimal digits are lost (the values were _rounded_).

In [95]:
'4G⊂ ZZZ9⊃'⎕FMT nm

21 1535 375 705
 1125 1964 464 1438
 797 1569 157 887

In this new example small values are not padded by zeroes, but by blanks.
It is also worth noting that we use `'ZZZ9'` instead of `'ZZZZ'` so that a single `'0'` gets printed in case there's any zeroes in the right argument:

In [109]:
'G⊂Z⊃' ⎕FMT 0

In [110]:
'G⊂9⊃' ⎕FMT 0

0

Decimal digits can be displayed only if we convert the values into integers and insert "artificial" decimal points between the `'Z'`'s or `'9'`'s:

In [114]:
'G⊂ ZZZ9.99⊃' ⎕FMT 100×nm

21.21 1534.88 375.46 704.50
 1125.14 1963.52 464.45 1438.25
 796.53 1569.00 157.14 886.59

Finally, we illustrate what we said earlier about the fact that symbols placed between descriptors `'Z'` and `'9'` are displayed only if they are surrounded by digits:

In [117]:
'G⊂Value ZZ-ZZ/Z9⊃' ⎕FMT 8 15 654 3852 19346 621184

Value 8
Value 15
Value 6/54
Value 38/52
Value 1-93/46
Value 62-11/84

This characteristic is useful when displaying numbers according to national conventions:

In [119]:
'G⊂ZZZ,ZZZ,ZZ9.99⊃' ⎕FMT 32145698710 8452 95732 64952465

321,456,987.10
 84.52
 957.32
 649,524.65

The above is the Anglo-American presentation and the example below is the French presentation:

In [120]:
'G⊂ZZZ ZZZ ZZ9,99⊃' ⎕FMT 32145698710 8452 95732 64952465

321 456 987,10
 84,52
 957,32
 649 524,65

Here is a surprising example:

In [123]:
'G⊂Simon ZZ Garfunkel ZZ⊃' ⎕FMT 4562.31 8699.84

Simon 45 Garfunkel 62
Simon 87

Here is the explanation of this surprising example:

 - the two numbers have been rounded like this: `4562 8700`;
 - because vectors are shown in columns, we get one printed line per item in the vector;
 - on the first line, `4562` has been split into `45` and `62`;
 - on the second line, `8700` has been split into `87` and `00`;
 - because we used a descriptor `'Z'`, these zeroes have been replaced by blanks, which means the characters that compose `'Garfunkel'` are no longer between non-blank digits and, therefore, are not reproduced.

#### Specification T

Specification `'X'` was used to specify an offset between a field and its neighbour.
Specification `'T'` (where `'T'` stands for tabular) specifies a position from the left margin.
This makes it easy to position data in a sheet.

In [128]:
ruler 'I2,T15,5A1,T30,4I6' ⎕FMT (75 91 34) chemistry nm

75 H2SO4 21 1535 375 705
91 CaCO3 1125 1964 464 1438
34 Fe2O3 797 1569 157 887
¯¯¯¯'¯¯¯¯|¯¯¯¯'¯¯¯¯|¯¯¯¯'¯¯¯¯|¯¯¯¯'¯¯¯¯|¯¯¯¯'¯¯¯¯|¯¯¯

As you can see, `chemistry` starts at the 15th position, and the first column of `nm` starts at the 30th position but, because it occupies 6 characters per column, it is right-aligned at the 35th character.

#### Specification K

Specification `'Kn'` is used to multiply a value by $10^n$ before it is displayed.
Here is how a numerical value is displayed normally:

'F12.2' ⎕FMT 123.45

Now we multiply the number by 1000 before displaying it:

'K3F12.2' ⎕FMT 123.45

Now we divide the number by 10 (and round it) before displaying it:

'K¯1F12.2' ⎕FMT 123.45

Because specification `'G'` only displays integer values, it is often convenient to use specification `'K'` to multiply decimal values by a power of 10 to obtain the correct display, as exemplified here:

'G⊂ZZZ ZZZ ZZ9.99⊃'⎕FMT 75435.39 66054.17 7.2 1673.08

Notice how the presentation above gives the wrong idea about the values.
However, if we were to use `'K2'` to multiply the values by 100 before displaying them, we would produce the correct representation:

'K2G⊂ZZZ ZZZ ZZ9.99⊃'⎕FMT 75435.39 66054.17 7.2 1673.08

<!--begin tip-->
***Tip***:

 > In the use-case above, where we use the specification `'Kn'` before using the specification `'G'`, the number of decimal places you use in the text descriptor of the specification `'G'` is the number `'n'` you should use with the specification `'Kn'`.
<!--end-->

For example, you would probably want to use `K3G⊂ZZ9.999⊃` or `K1G⊂ZZ9.9⊃` to format small numbers with 3 and 1 decimal places, respectively.

#### Specification E

This specification in the left argument to `⎕FMT` is used to display numeric values, but in scientific (or **e**xponential) form.

Its syntax is very similar to the syntax of the specification `'F'`:

 - `'rEw.s'`
   - `'w'` is the **w**idth (in number of characters) dedicated to each column of numbers;
   - `'s'` is the number of **s**ignificant digits displayed in the mantissa; and
   - `'r'` is the **r**epetition factor.

In [161]:
ruler 'E12.4' ⎕FMT 12553 0.0487 ¯62.133

1.255E4 
 4.870E¯2 
 ¯6.213E1 
¯¯¯¯'¯¯¯¯|¯¯

You can see that each number is represented by 12 characters, with exactly 4 significant digits.
However, in order to make room for larger exponents, the last column is left blank.

You can refer to [Wikipedia's article on "Scientific Notation"](https://en.wikipedia.org/wiki/Scientific_notation#E_notation) for more information on how scientific notation works.

### Qualifiers and Affixtures

Specifications `'I'`, `'F'`, and `'G'` can be associated with _qualifiers_ and _affixtures_:

 - _qualifiers_ modify the presentation of numeric values; and
 - _affixtures_ print additional characters when some conditions are satisfied.

_Qualifiers_ and _affixtures_ must be specified to the **left** of the specification they modify.

#### Qualifiers

 - `'Kn'` multiplies numbers by $10^n$ before displaying them;
 - `'B'` replaces zero values by **B**lanks;
 - `'C'` separates triads of characters by **C**ommas in the integer part of a number;
 - `'L'` aligns the value to the **L**eft of its field;
 - `'Z'` fills up the left part of the zone reserved for a field with **Z**eroes;
 - `'Ov⊂text⊃'` replaces **O**nly the specific value `v` with the given `text`. If omitted, `v` is assumed to be zero, so replacing zeroes with a special text string is easy; and
 - `'S⊂os⊃'` replaces characters with **S**ubstitution characters. `os` is a list of couples of characters, where
   - `o` is the **o**riginal character; and
   - `s` is the **s**ubstitute character.

From the qualifiers above, `'C'`, `'L'`, and `'Z'` do **not** work with specification `'G'`.
Furthermore, note that the specification `'S⊂os⊃'` applies only to the replacement of the following characters:
 
 - `.` – the decimal separator;
 - `,` – the thousands separator produced by qualifier `'C'`;
 - `*` – the overflow character used when a value takes up more space than the space available for it;
 - `0` – the fill character produced by qualifier `'Z'`; and
 - `_` – the character indicating lack of precision (see [the Specialist's Section about it](#Lack-of-Precision)).

#### Examples of Qualifiers

Multiply a number by 1000 before displaying it:

In [192]:
'K3F12.2' ⎕FMT 123.45

123450.00

Now divide a number by 10 (and round it) before displaying it:

In [193]:
'K¯1F12.2' ⎕FMT 123.45

12.35

Because specification `'G'` only displays integer values, it is often convenient to use specification `'K'` to multiply decimal values by a power of 10 to obtain the correct display, as exemplified here:

In [198]:
'G⊂ZZ9.99⊃'⎕FMT 435.39 54.17 7.2 673.08

4.35
 0.54
 0.07
 6.73

Notice how the presentation above gives the wrong idea about the values.
However, if we were to use `'K2'` to multiply the values by 100 before displaying them, we would produce the correct representation:

In [199]:
'K2G⊂ZZ9.99⊃'⎕FMT 435.39 54.17 7.2 673.08

435.39
 54.17
 7.20
673.08

<!--begin tip-->
***Tip***:

 > In the use-case above, the number of decimal places you use in the text descriptor of the specification `'G'` is the number `'n'` you should use with the qualifier `'Kn'`.
<!--end-->

Now we use the qualifier `'Z'` to pad values with zeroes on the left.
Notice, also, how a repetition factor may be applied to a list of specifications enclosed by parentheses:

In [202]:
'ZI2,2(⊂/⊃,ZI2)' ⎕FMT 1 3⍴9 7 98

09/07/98

We can reproduce the Anglo-American and French representations for large numbers (that we produced with specification `'G'`) with the `'C'` and `'S'` qualifiers:

In [210]:
'CF13.2' ⎕FMT 74815926.03

74,815,926.03

In [213]:
'S⊂, .,⊃CF13.2' ⎕FMT 74815926.03

74 815 926,03

In this last example, commas have been replaced by blanks and the decimal point has been substituted by a comma, using the `'S'` qualifier, to obtain a French presentation.

For the next examples, let us use the following numeric matrix:

In [214]:
⎕← yop ← 2 3⍴178.23 0 ¯87.64 0 ¯681.19 42

178.23 0 ¯87.64
 0 ¯681.19 42

Using `'B'` we can replace zeroes with blanks:

In [215]:
'BF9.2' ⎕FMT yop

178.23 ¯87.64
 ¯681.19 42.00

This is the same thing as using `'O'` to only replace zeroes with blanks:

In [222]:
'O⊂⊃F9.2' ⎕FMT yop

178.23 ¯87.64
 ¯681.19 42.00

Recall that `'O'` assumes, by default, that we want to replace zeroes with the text within `⊂ ⊃`.
If we wanted, we could replace `42` by blanks, instead:

In [223]:
'O42⊂⊃F9.2' ⎕FMT yop

178.23 0.00 ¯87.64
 0.00 ¯681.19

We can also replace the value(s) by something other than blanks.
For that, we just specify the replacement text inside the text delimiters:

In [224]:
'O⊂none⊃F9.2' ⎕FMT yop

178.23 none ¯87.64
 none ¯681.19 42.00

In [231]:
'O42⊂2+4×10⊃F9.2' ⎕FMT yop

178.23 0.00 ¯87.64
 0.00 ¯681.19 2+4×10

Using `'L'` we can align all the values on the left instead:

In [245]:
'LO42⊂2+4×10⊃F9.2' ⎕FMT yop

178.23 0.00 ¯87.64 
0.00 ¯681.19 2+4×10

If we use `Z`, then we can fill the whole field with zeroes:

In [243]:
'I4' ⎕FMT 42

42

In [244]:
'ZI4' ⎕FMT 42

0042

#### Affixtures

 - `'M⊂text⊃'` replaces the **M**inus sign to the **left** of negative values with the given `text`;
 - `'N⊂text⊃'` adds the given `text` to the **right** of **n**egative values;
 - `'P⊂text⊃'` adds the given `text` to the **left**  of **p**ositive values;
 - `'Q⊂text⊃'` adds the given `text` to the **right** of positive values; and
 - `'R⊂text⊃'` **r**epeats the given `text` as many times as necessary to fill the printing zone entirely, before the digits are overlaid on top. Positions which are not occupied by the digits allow the text to appear. In other words, the `text` will act as a background for the formatted value.

It is important to note that the width of the text added by an _affixture_ must be accounted for in the total width reserved for the column.

#### Examples of Affixtures

It is a common accounting practice to represent negative numbers surrounded by parentheses, instead of with a minus sign.
We can easily format numbers to abide by this practice using `'M⊂(⊃'` to replace the minus sign of negative numbers with `'('` and using `'N⊂)⊃'` to add a `')'` to the right of negative numbers:

In [248]:
'M⊂(⊃N⊂)⊃F10.2' ⎕FMT yop

178.23 0.00 (87.64)
 0.00 (681.19) 42.00

However, we can see that the negative values are no longer aligned with the positive ones.
We can fix this by using the affixture `'Q'` to add a blank to the right of positive values:

In [249]:
'Q⊂ ⊃M⊂(⊃N⊂)⊃F10.2' ⎕FMT yop

178.23 0.00 (87.64)
 0.00 (681.19) 42.00

Now, instead of surrounding negative numbers with parentheses, suppose we want to represent them with a `'-'` instead of a `'¯'`, and also add a `'+'` to the left of the positive numbers.
For that, we just reuse `'M'` and introduce `'P'`:

In [256]:
'P⊂+⊃M⊂-⊃F10.2' ⎕FMT yop

+178.23 +0.00 -87.64
 +0.00 -681.19 +42.00

Finally, to illustrate how the affixture `'R'` works, we will draw the representation of `yop` in a matrix of `'⎕'`'s:

In [258]:
'R⊂⎕⊃F10.2' ⎕FMT yop

⎕⎕⎕⎕178.23⎕⎕⎕⎕⎕⎕0.00⎕⎕⎕⎕¯87.64
⎕⎕⎕⎕⎕⎕0.00⎕⎕⎕¯681.19⎕⎕⎕⎕⎕42.00

Notice that this is just an overlay of

In [259]:
'F10.2' ⎕FMT yop

178.23 0.00 ¯87.64
 0.00 ¯681.19 42.00

on top of

In [262]:
(⍴'F10.2' ⎕FMT yop)⍴'⎕'

⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕
⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

#### Remarks on Qualifiers and Affixtures

 - Qualifiers and affixtures can be cumulated and can be placed in any order.

For example,

In [268]:
'S⊂, .,⊃CF10.2' ⎕FMT nm

21,21 1 534,88 375,46 704,50
 1 125,14 1 963,52 464,45 1 438,25
 796,53 1 569,00 157,14 886,59

In [269]:
'CS⊂, .,⊃F10.2' ⎕FMT nm

21,21 1 534,88 375,46 704,50
 1 125,14 1 963,52 464,45 1 438,25
 796,53 1 569,00 157,14 886,59

 - Blanks can be inserted between specifications, qualifiers, and affixtures.

For example,

In [270]:
'S⊂, .,⊃CF10.2' ⎕FMT nm

21,21 1 534,88 375,46 704,50
 1 125,14 1 963,52 464,45 1 438,25
 796,53 1 569,00 157,14 886,59

In [272]:
'S⊂, .,⊃ C F10.2' ⎕FMT nm

21,21 1 534,88 375,46 704,50
 1 125,14 1 963,52 464,45 1 438,25
 796,53 1 569,00 157,14 886,59

 - A repetition factor can apply to a group of descriptors placed between parentheses, like we have seen above.

We replicate a similar example here:

In [275]:
'ZI2,2(⊂/⊃,ZI2)' ⎕FMT 1 3⍴5 3 98

05/03/98

 - Various errors may occur, all signalled by the message `'FORMAT ERROR'`. Here are some frequent errors:
   - a numeric value is matched with an `'A'` specification;
   - character data is matched with a specification other than `'A'`;
   - the format specification is ill-shaped (in that case, check that delimiters and parentheses are well balanced);
   - in decimal specifications (`'F'` and `'E'`), the specified width is too small, so the decimal digits cannot be represented; and
   - you are using incompatible qualifiers and/or specifications (for example, mixing `'L'` with `'Z'` in the same specification).

![Dyalog APL programming is pure Art.](res/Programming_Art.png)

## The Specialist's Section

<br />
<center><i>Each chapter is followed by a "Specialist's Section" like this one. This section is dedicated to skilled APLers, who wish to improve their knowledge.</i>

You will find here rare or complex usages of the concepts presented in this chapter, or discover extended explanations which need the knowledge of some symbols that will be seen much further in the book.

<b>If you are exploring APL for the first time, skip this section and go to the next chapter.</b></center>

### Execute

#### Name Conflict

Suppose that we would like to switch two letters inside a word.
Let us write a function which accepts the index of the letters to switch as its left argument, and the **name** of an existing variable as its right argument.

It is important to note that the function works on the **name** of the variable, not on its **value**:

In [278]:
]dinput
Exchange ← {
    v ← ⌽⍺
    text ← ⍵,'[⍺]←',⍵,'[v]'
    ⍎text
}

Now we create a character vector containing a word, call our function to exchange two characters, and then check the value of the variable again:

In [283]:
word ← 'moral'
3 5 Exchange 'word'
word

molar

In our example, `⍵` was equal to `'word'`, so the statement that assigns to `text` was equivalent to `text ← 'word[⍺]←word[v]'`.

All is well up to now.
But now, let us try again on another variable:

In [287]:
v ← 'rats'
1 2 Exchange 'v'
v

rats

The result now is **wrong**, it should have been `'arts'`.
The reason is that the assignment to `text` is equivalent to `text ← 'v[⍺]←v[v]'`, but `v` is the name of a local variable in the function.
When the last statement is executed, it exchanges items inside the **local** variable, not in the global one, which remains unchanged!

In fact, if we call `Exchange` with larger indices, we get an `INDEX ERROR` because the local variable `v` only has two elements:

In [289]:
1 4 Exchange 'v'

⍎INDEX ERROR
Exchange[3] v[⍺]←v[v]
                  ∧


In other words, be careful when _execute_ is expected to work on global names, as there may be a risk of conflict with local names.
In order to reduce this risk, programmers sometimes use complex and weird names for the local names in such functions.
Another alternative is to try and write your functions to not rely on `⍎` for this kind of things, as that creates code that is much harder to debug; instead, write functions that take arguments and return their results explicitly.
See, for example, [the subsection “Why Should a Function Return a Result?”](./User-Defined-Functions.ipynb#Why-Should-a-Function-Return-a-Result?).